In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_store_sku_pilot_machine_learning 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
WITH list_store AS (
    SELECT store_id, 'CONTROL' AS scenario
    FROM VALUES
    ('5971'), ('3812'), ('3357'), ('3800'), ('4120'), ('2AE1'), ('6678'), ('6547'), ('2AG1'), ('6938'),
    ('6851'), ('5626'), ('6096'), ('6654'), ('4195'), ('2A85'), ('4471'), ('6934'), ('2AC7'), ('4182'),
    ('6266'), ('6145'), ('3892'), ('4092'), ('3855'), ('6953'), ('6839')
    AS t(store_id)
    UNION ALL
    SELECT store_id, 'ML' AS scenario
    FROM VALUES
    ('4318'), ('4240'), ('4084'), ('6549'), ('3579'), ('4310'), ('4299'), ('6475'), ('6918'), ('6458'),
    ('6536'), ('4181'), ('4401'), ('4472'), ('3798'), ('5756'), ('6034'), ('5198'), ('3770'), ('4209'),
    ('3671'), ('6773'), ('6842'), ('6943'), ('6113'), ('6234'), ('6472')
    AS t(store_id)
    UNION ALL
    SELECT store_id, 'ML_INV_POLICY' AS scenario
FROM VALUES
  ('4485'), ('3669'), ('3919'), ('5212'), ('6582'), ('3847'), ('5419'), ('6715'), ('2A01'), ('6928'),
  ('5330'), ('3808'), ('5194'), ('3920'), ('4204'), ('4228'), ('6693'), ('4399'), ('6002'), ('5776'),
  ('4074'), ('3780'), ('3427'), ('2A19'), ('6716'), ('6290')
AS t(store_id)
),
main AS (
    SELECT 
        a.store_id, a.product_id,
        b.scenario
    FROM {catalog_name}.udp_wcm_gold_vcm_db_inventory.d_mdq_store_dc_sku_biweekly a
    INNER JOIN list_store b
        ON a.store_id = b.store_id
    LEFT JOIN {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_new_winare_store_sku_golive c
        ON a.store_id = c.store_id AND a.product_id = c.product_id
    LEFT JOIN {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_store_sku_winare_promotion_golive d
        ON a.product_id = d.product_id
    WHERE a.city = 'BÌNH DƯƠNG'
        AND a.dc_id = '1312'
        AND c.store_id IS NULL
        AND d.product_id IS NULL
        AND IFNULL(a.mdq, 0) > 0
)
SELECT *
FROM main;
""")